### Let's try some video / animation around matplotlib and opencv

The first, plot a empty figure

In [ ]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt

from matplotlib import animation, rc
from IPython.display import HTML

# First set up the figure, the axis, and the plot element we want to animate
fig, ax = plt.subplots()

ax.set_xlim(( 0, 2))
ax.set_ylim((-2, 2))

line, = ax.plot([], [], lw=2)

Then add x and y one dimension array data, and shift data array per 20 ms for animation effect. One trick is to wrap the animation as HTML5 video for notebook rendering. 

In [ ]:
# initialization function: plot the background of each frame
def init():
    line.set_data([], [])
    return (line,)

# animation function. This is called sequentially
def animate(i):
    x = np.linspace(0, 2, 1000)
    y = np.sin(2 * np.pi * (x - 0.01 * i))
    line.set_data(x, y)
    return (line,)

# call the animator. blit=True means only re-draw the parts that have changed.
anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=100, interval=20, blit=True)

HTML(anim.to_html5_video())

Ok, after single dimension scalar data animation, let's try two dimension array animation. At first, we need create a figure with canvas ready. 

In [ ]:
fig = plt.figure()

def f(x, y):
    return np.sin(x) + np.cos(y)

x = np.linspace(0, 2 * np.pi, 120)
y = np.linspace(0, 2 * np.pi, 100).reshape(-1, 1)

canvas = plt.imshow(f(x, y), animated=True)

Again, shift image array data and animate it. 

In [ ]:
def updatefig(*args):
    global x, y
    x += np.pi / 15.
    y += np.pi / 20.
    canvas.set_array(f(x, y))
    return (canvas,)

ani = animation.FuncAnimation(fig, updatefig, interval=50, blit=True)
HTML(ani.to_html5_video())

### See how we could hook opencv into matplot on video data. 

Import OpenCV module, and load a sample short video file, check its resolution and render first frame.

In [ ]:
import cv2

cap = cv2.VideoCapture('data/sample.mp4')
vh = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
vw = cap.get(cv2.CAP_PROP_FRAME_WIDTH)

print("Video resolution is (%d x %d)" % (vw, vh)) 

fig = plt.figure()

ret, frame = cap.read()
rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

canvas = plt.imshow(rgb, animated=True)

Load other video frames into animation. It's possible to specify number of frames to cache by save_count of FuncAnimation.

In [ ]:
def updatefig(*args):
    global frame, rgb
    ret, frame = cap.read()
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    canvas.set_array(rgb)
    return (canvas,)

ani = animation.FuncAnimation(fig, updatefig, interval=50, blit=True)
HTML(ani.to_html5_video())

Be certain to close and release the video file

In [ ]:
cap.release()